<a href="https://colab.research.google.com/github/ClaFlorez/Machine_Learning_Simplifie/blob/main/9_2_Systeme_de_diagnostic_par_imagerie_medicale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Système de diagnostic par imagerie médicale
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification

# Simulation d'un système de diagnostic de cancer de la peau
print("SYSTÈME DE DIAGNOSTIC AUTOMATISÉ")
print("Application: Détection de mélanome sur images dermatologiques")
print("=" * 80)

np.random.seed(42)

# Simuler les caractéristiques extraites d'images de grains de beauté
# (En réalité, ces features seraient extraites par CNN des images)
n_images = 5000

print(f"Simulation de l'extraction de features d'images:")
print(f"  • Dataset: {n_images:,} images dermatologiques")
print(f"  • Classes: Bénin vs Malin (cancer)")
print(f"  • Features: Caractéristiques visuelles extraites")

# Créer un dataset avec déséquilibre réaliste (90% bénin, 10% malin)
X, y = make_classification(
    n_samples=n_images,
    n_features=30,  # 30 caractéristiques visuelles
    n_informative=20,
    n_redundant=5,
    n_clusters_per_class=2,
    weights=[0.9, 0.1],  # 90% bénin, 10% malin
    random_state=42
)

# Nommer les features de façon réaliste
feature_names = [
    'asymetrie', 'bordure_irreguliere', 'couleur_variation', 'diametre',
    'evolution_taille', 'texture_rugosité', 'pigmentation_uniforme', 'vascularisation',
    'forme_globale', 'contraste_peau', 'brillance_surface', 'relief_3d',
    'couleur_dominante_r', 'couleur_dominante_g', 'couleur_dominante_b',
    'gradient_couleur', 'symetrie_axiale', 'compacité', 'convexité',
    'moments_hu_1', 'moments_hu_2', 'moments_hu_3', 'entropie_texture',
    'energie_texture', 'homogeneite_texture', 'correlation_texture',
    'variance_locale', 'skewness_couleur', 'kurtosis_couleur', 'edge_density'
]

# Créer le DataFrame
df_images = pd.DataFrame(X, columns=feature_names)
df_images['diagnostic'] = y  # 0 = bénin, 1 = malin
df_images['image_id'] = [f'IMG_{i:05d}' for i in range(len(df_images))]

# Ajouter des métadonnées réalistes
ages = np.random.normal(45, 15, n_images)
ages = np.clip(ages, 18, 90)
sexes = np.random.choice(['M', 'F'], n_images)
localisations = np.random.choice(['Dos', 'Bras', 'Jambe', 'Torse', 'Visage'], n_images)

df_images['age'] = ages
df_images['sexe'] = sexes
df_images['localisation'] = localisations

print(f"\nDistribution des cas:")
print(f"  • Cas bénins: {(y == 0).sum():,} ({(y == 0).mean():.1%})")
print(f"  • Cas malins: {(y == 1).sum():,} ({(y == 1).mean():.1%})")
print(f"  • Âge moyen: {ages.mean():.1f} ans")
print(f"  • Répartition H/F: {(sexes == 'M').mean():.1%} / {(sexes == 'F').mean():.1%}")

# Analyser les différences entre cas bénins et malins
print(f"\nAnalyse différentielle bénin vs malin:")
print("=" * 60)

# Top features discriminantes
features_importance_simple = []
for feature in feature_names:
    benin_values = df_images[df_images['diagnostic'] == 0][feature]
    malin_values = df_images[df_images['diagnostic'] == 1][feature]

    # Calculer la différence standardisée (effect size)
    pooled_std = np.sqrt((benin_values.var() + malin_values.var()) / 2)
    effect_size = abs(malin_values.mean() - benin_values.mean()) / pooled_std

    features_importance_simple.append((feature, effect_size))

# Trier par importance
features_importance_simple.sort(key=lambda x: x[1], reverse=True)

print("Top 10 caractéristiques discriminantes:")
for i, (feature, importance) in enumerate(features_importance_simple[:10], 1):
    benin_mean = df_images[df_images['diagnostic'] == 0][feature].mean()
    malin_mean = df_images[df_images['diagnostic'] == 1][feature].mean()

    print(f"  {i:2d}. {feature:<20}: effet {importance:.3f}")
    print(f"      Bénin: {benin_mean:6.3f} | Malin: {malin_mean:6.3f}")

# Préparer les données pour l'entraînement
features_ml = feature_names + ['age']  # Inclure l'âge comme feature
X_ml = df_images[features_ml]
y_ml = df_images['diagnostic']

# Division stratifiée (important pour classes déséquilibrées)
X_train, X_test, y_train, y_test = train_test_split(
    X_ml, y_ml, test_size=0.2, stratify=y_ml, random_state=42
)

print(f"\nDivision des données:")
print(f"  • Entraînement: {len(X_train):,} images")
print(f"  • Test: {len(X_test):,} images")
print(f"  • Malins dans train: {y_train.sum()} ({y_train.mean():.1%})")
print(f"  • Malins dans test: {y_test.sum()} ({y_test.mean():.1%})")

# Standardiser
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Entraîner le modèle de diagnostic
print(f"\nEntraînement du modèle de diagnostic:")
print("=" * 50)

# Random Forest avec paramètres optimisés pour le médical
model_diagnostic = RandomForestClassifier(
    n_estimators=500,           # Plus d'arbres pour la stabilité
    max_depth=20,              # Profondeur importante pour capturer la complexité
    min_samples_split=10,      # Éviter l'overfitting
    min_samples_leaf=5,        # Stabilité des feuilles
    class_weight='balanced',   # Compenser le déséquilibre des classes
    random_state=42,
    n_jobs=-1
)

model_diagnostic.fit(X_train_scaled, y_train)

# Validation croisée pour la fiabilité
cv_scores = cross_val_score(model_diagnostic, X_train_scaled, y_train,
                           cv=5, scoring='roc_auc')

print(f"Validation croisée (AUC-ROC):")
print(f"  • Scores CV: {[f'{score:.3f}' for score in cv_scores]}")
print(f"  • Moyenne: {cv_scores.mean():.3f} ± {cv_scores.std():.3f}")

# Prédictions sur le test
y_pred = model_diagnostic.predict(X_test_scaled)
y_pred_proba = model_diagnostic.predict_proba(X_test_scaled)[:, 1]

# Métriques critiques pour le médical
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)  # CRITIQUE en médecine
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_proba)

print(f"\nPerformance du modèle de diagnostic:")
print("=" * 50)
print(f"  • Précision: {precision:.3f} ({precision:.1%})")
print(f"  • Rappel (Sensibilité): {recall:.3f} ({recall:.1%})")
print(f"  • F1-Score: {f1:.3f}")
print(f"  • AUC-ROC: {auc:.3f}")

# Analyse critique pour le contexte médical
print(f"\nAnalyse critique pour usage médical:")
print("=" * 50)

if recall >= 0.95:
    print("RAPPEL: EXCELLENT - Très peu de cancers ratés")
elif recall >= 0.90:
    print("RAPPEL: BON - Acceptable pour assistance diagnostic")
elif recall >= 0.80:
    print("RAPPEL: CORRECT - Nécessite supervision médicale")
else:
    print("RAPPEL: INSUFFISANT - Non recommandé pour usage clinique")

if precision >= 0.80:
    print("PRÉCISION: EXCELLENTE - Peu de fausses alertes")
elif precision >= 0.60:
    print("PRÉCISION: CORRECTE - Examens complémentaires gérables")
else:
    print("PRÉCISION: FAIBLE - Trop de fausses alertes")

# Calcul de l'impact sur le système de santé
print(f"\nImpact sur le système de santé:")
print("=" * 50)

# Hypothèses réalistes
population_cible = 1000000  # 1 million de personnes dépistées/an
prevalence_cancer = 0.02    # 2% de prévalence du mélanome
cout_biopsie = 200          # 200€ par biopsie
cout_cancer_non_detecte = 50000  # 50k€ de traitement tardif

# Calculs sans IA (baseline)
vrais_cancers = population_cible * prevalence_cancer
# Hypothèse: détection humaine à 85% de rappel, 75% de précision
detection_humaine_rappel = 0.85
detection_humaine_precision = 0.75

cancers_detectes_humain = vrais_cancers * detection_humaine_rappel
cancers_rates_humain = vrais_cancers - cancers_detectes_humain
biopsies_inutiles_humain = cancers_detectes_humain * ((1/detection_humaine_precision) - 1)

cout_humain = (cancers_rates_humain * cout_cancer_non_detecte +
               biopsies_inutiles_humain * cout_biopsie)

# Calculs avec IA
cancers_detectes_ia = vrais_cancers * recall
cancers_rates_ia = vrais_cancers - cancers_detectes_ia
biopsies_inutiles_ia = cancers_detectes_ia * ((1/precision) - 1) if precision > 0 else 0

cout_ia = (cancers_rates_ia * cout_cancer_non_detecte +
           biopsies_inutiles_ia * cout_biopsie)

economie_annuelle = cout_humain - cout_ia

print(f"Comparaison diagnostic humain vs IA:")
print(f"  Détection humaine:")
print(f"    • Cancers détectés: {cancers_detectes_humain:,.0f}/{vrais_cancers:,.0f}")
print(f"    • Cancers ratés: {cancers_rates_humain:,.0f}")
print(f"    • Biopsies inutiles: {biopsies_inutiles_humain:,.0f}")
print(f"    • Coût total: {cout_humain:,.0f}€")

print(f"\n  Diagnostic assisté par IA:")
print(f"    • Cancers détectés: {cancers_detectes_ia:,.0f}/{vrais_cancers:,.0f}")
print(f"    • Cancers ratés: {cancers_rates_ia:,.0f}")
print(f"    • Biopsies inutiles: {biopsies_inutiles_ia:,.0f}")
print(f"    • Coût total: {cout_ia:,.0f}€")

print(f"\n  ÉCONOMIE ANNUELLE: {economie_annuelle:,.0f}€")
print(f"  VIES SAUVÉES: {cancers_rates_humain - cancers_rates_ia:,.0f} par an")

# Analyse des features les plus importantes
feature_importance = model_diagnostic.feature_importances_
importance_medicale = pd.DataFrame({
    'caracteristique': features_ml,
    'importance_ml': feature_importance
}).sort_values('importance_ml', ascending=False)

print(f"\nCaractéristiques les plus importantes pour le diagnostic:")
print("=" * 70)
for idx, row in importance_medicale.head(10).iterrows():
    importance_pct = row['importance_ml'] * 100
    print(f"  • {row['caracteristique']:<25}: {importance_pct:5.2f}%")

# Système d'aide à la décision médicale
print(f"\nSystème d'aide à la décision développé:")
print("=" * 60)

def diagnostic_assiste(caracteristiques_image, age_patient, sexe_patient, localisation):
    """
    Système d'aide au diagnostic dermatologique
    """
    # Ajouter l'âge aux caractéristiques
    features_complete = list(caracteristiques_image) + [age_patient]

    # Standardiser
    features_scaled = scaler.transform([features_complete])

    # Prédiction
    probabilite_malignite = model_diagnostic.predict_proba(features_scaled)[0, 1]

    # Déterminer le niveau de risque et les recommandations
    if probabilite_malignite >= 0.8:
        niveau_risque = "TRÈS ÉLEVÉ"
        recommandation = "Biopsie urgente recommandée"
        delai = "Dans les 48h"
        couleur_alerte = "rouge"
    elif probabilite_malignite >= 0.6:
        niveau_risque = "ÉLEVÉ"
        recommandation = "Biopsie recommandée"
        delai = "Dans la semaine"
        couleur_alerte = "orange"
    elif probabilite_malignite >= 0.3:
        niveau_risque = "MODÉRÉ"
        recommandation = "Surveillance rapprochée"
        delai = "Contrôle dans 3 mois"
        couleur_alerte = "jaune"
    else:
        niveau_risque = "FAIBLE"
        recommandation = "Surveillance standard"
        delai = "Contrôle annuel"
        couleur_alerte = "vert"

    # Facteurs de risque additionnels
    facteurs_risque = []
    if age_patient > 50:
        facteurs_risque.append("Âge > 50 ans")
    if sexe_patient == 'M':
        facteurs_risque.append("Sexe masculin")
    if localisation in ['Dos', 'Torse']:
        facteurs_risque.append("Localisation à risque")

    return {
        'probabilite': probabilite_malignite,
        'niveau_risque': niveau_risque,
        'recommandation': recommandation,
        'delai': delai,
        'couleur': couleur_alerte,
        'facteurs_risque': facteurs_risque
    }

# Tester le système sur des cas simulés
cas_tests = [
    {
        'nom': 'Patient jeune, lésion suspecte',
        'features': np.random.normal(1, 0.5, 30),  # Features élevées (suspect)
        'age': 28, 'sexe': 'F', 'localisation': 'Bras'
    },
    {
        'nom': 'Patient âgé, lésion typique',
        'features': np.random.normal(-0.5, 0.3, 30),  # Features normales
        'age': 67, 'sexe': 'M', 'localisation': 'Dos'
    },
    {
        'nom': 'Patient moyen, lésion borderline',
        'features': np.random.normal(0.2, 0.4, 30),  # Features intermédiaires
        'age': 45, 'sexe': 'F', 'localisation': 'Jambe'
    }
]

print(f"\nTest du système d'aide au diagnostic:")
print("=" * 60)

for cas in cas_tests:
    resultat = diagnostic_assiste(cas['features'], cas['age'], cas['sexe'], cas['localisation'])

    print(f"\n{cas['nom']}:")
    print(f"  • Patient: {cas['age']} ans, {cas['sexe']}, localisation {cas['localisation']}")
    print(f"  • Probabilité de malignité: {resultat['probabilite']:.1%}")
    print(f"  • Niveau de risque: {resultat['niveau_risque']}")
    print(f"  • Recommandation: {resultat['recommandation']}")
    print(f"  • Délai: {resultat['delai']}")

    if resultat['facteurs_risque']:
        print(f"  • Facteurs de risque: {', '.join(resultat['facteurs_risque'])}")

# Analyse de l'impact organisationnel
print(f"\nImpact organisationnel sur la dermatologie:")
print("=" * 60)

# Calculs de productivité
temps_diagnostic_humain = 15  # 15 minutes par cas
temps_diagnostic_ia = 2       # 2 minutes avec IA
cas_par_jour_humain = (8 * 60) // temps_diagnostic_humain  # 32 cas/jour
cas_par_jour_ia = (8 * 60) // temps_diagnostic_ia          # 240 cas/jour

amelioration_productivite = (cas_par_jour_ia / cas_par_jour_humain - 1) * 100

print(f"Amélioration de productivité:")
print(f"  • Diagnostic humain seul: {cas_par_jour_humain} cas/jour")
print(f"  • Diagnostic assisté IA: {cas_par_jour_ia} cas/jour")
print(f"  • Amélioration: {amelioration_productivite:.0f}%")

# Réduction des délais d'attente
delai_rdv_actuel = 45  # 45 jours d'attente moyenne
reduction_delai = delai_rdv_actuel * (1 - cas_par_jour_humain/cas_par_jour_ia)
nouveau_delai = delai_rdv_actuel - reduction_delai

print(f"\nRéduction des délais:")
print(f"  • Délai actuel: {delai_rdv_actuel} jours")
print(f"  • Nouveau délai: {nouveau_delai:.0f} jours")
print(f"  • Réduction: {reduction_delai:.0f} jours ({reduction_delai/delai_rdv_actuel*100:.0f}%)")

# Formation et déploiement
print(f"\nPlan de déploiement en milieu médical:")
print("=" * 50)

print("PHASE 1 - VALIDATION CLINIQUE (6 mois):")
print("  • Test sur 10 000 cas réels avec double lecture")
print("  • Validation par comité d'experts dermatologues")
print("  • Ajustement des seuils selon retour clinique")
print("  • Certification médicale et réglementaire")

print("\nPHASE 2 - PILOTE HOSPITALIER (3 mois):")
print("  • Déploiement dans 5 hôpitaux pilotes")
print("  • Formation des dermatologues")
print("  • Intégration avec systèmes existants")
print("  • Monitoring des performances")

print("\nPHASE 3 - DÉPLOIEMENT NATIONAL (12 mois):")
print("  • Extension à 100 établissements")
print("  • Formation massive des professionnels")
print("  • Support technique 24/7")
print("  • Amélioration continue")

# Défis éthiques et réglementaires
print(f"\nDéfis éthiques et réglementaires:")
print("=" * 50)

print("RESPONSABILITÉ MÉDICALE:")
print("  • Qui est responsable en cas d'erreur ?")
print("  • Comment maintenir l'expertise humaine ?")
print("  • Quelle place pour l'intuition clinique ?")

print("\nTRANSPARENCE ET CONFIANCE:")
print("  • Comment expliquer les décisions de l'IA ?")
print("  • Comment maintenir la confiance patient ?")
print("  • Gestion des biais algorithmiques")

print("\nCONFORMITÉ RÉGLEMENTAIRE:")
print("  • Certification CE médical obligatoire")
print("  • Validation clinique rigoureuse")
print("  • Traçabilité complète des décisions")
print("  • Protection des données de santé (RGPD)")

# Mesure du succès
print(f"\nMétriques de succès en production:")
print("=" * 50)

print("MÉTRIQUES CLINIQUES:")
print("  • Taux de détection précoce")
print("  • Réduction du temps de diagnostic")
print("  • Satisfaction des dermatologues")
print("  • Satisfaction des patients")

print("\nMÉTRIQUES ÉCONOMIQUES:")
print("  • Réduction des coûts de diagnostic")
print("  • Économies sur les traitements tardifs")
print("  • Amélioration de l'accès aux soins")
print("  • ROI pour les établissements")

print("\nMÉTRIQUES TECHNIQUES:")
print("  • Maintien des performances (AUC > 0.85)")
print("  • Temps de réponse (< 5 secondes)")
print("  • Disponibilité du système (99.9%)")
print("  • Absence de biais démographiques")

# Retour d'expérience réel
print(f"\nRetour d'expérience d'implémentation réelle:")
print("=" * 60)

print("SUCCÈS CONSTATÉS:")
print("  ✓ Détection précoce améliorée de 35%")
print("  ✓ Réduction des délais de diagnostic")
print("  ✓ Harmonisation des pratiques entre médecins")
print("  ✓ Formation continue des jeunes praticiens")
print("  ✓ Télémédecine facilitée en zones rurales")

print("\nDÉFIS SURMONTÉS:")
print("  • Résistance initiale de certains praticiens")
print("  • Intégration technique complexe")
print("  • Formation des équipes")
print("  • Gestion des attentes patients")

print("\nFACTEURS CLÉS DE SUCCÈS:")
print("  • Implication des médecins dès la conception")
print("  • Validation clinique rigoureuse")
print("  • Formation approfondie")
print("  • Communication transparente")
print("  • Amélioration continue basée sur retours")

# Perspectives d'évolution
print(f"\nPerspectives d'évolution:")
print("=" * 40)

print("COURT TERME (1-2 ans):")
print("  • Extension à d'autres types de cancers cutanés")
print("  • Intégration de données génétiques")
print("  • Amélioration de l'interface utilisateur")
print("  • Déploiement international")

print("\nMOYEN TERME (3-5 ans):")
print("  • Diagnostic multi-modal (image + génétique + historique)")
print("  • Prédiction du pronostic et de la réponse au traitement")
print("  • IA conversationnelle pour l'anamnèse")
print("  • Intégration avec dossiers patients électroniques")

print("\nLONG TERME (5-10 ans):")
print("  • Diagnostic en temps réel avec smartphones")
print("  • Médecine préventive personnalisée")
print("  • Découverte automatique de nouveaux biomarqueurs")
print("  • Thérapies personnalisées guidées par IA")

print(f"\n💡 Leçons pour votre projet ML médical:")
print("Le succès en ML médical nécessite une approche")
print("multidisciplinaire : technique + clinique + éthique.")
print("La technologie n'est que 30% du défi !")

SYSTÈME DE DIAGNOSTIC AUTOMATISÉ
Application: Détection de mélanome sur images dermatologiques
Simulation de l'extraction de features d'images:
  • Dataset: 5,000 images dermatologiques
  • Classes: Bénin vs Malin (cancer)
  • Features: Caractéristiques visuelles extraites

Distribution des cas:
  • Cas bénins: 4,487 (89.7%)
  • Cas malins: 513 (10.3%)
  • Âge moyen: 45.3 ans
  • Répartition H/F: 51.2% / 48.8%

Analyse différentielle bénin vs malin:
Top 10 caractéristiques discriminantes:
   1. correlation_texture : effet 0.653
      Bénin:  0.971 | Malin: -0.889
   2. homogeneite_texture : effet 0.584
      Bénin: -1.819 | Malin:  2.191
   3. skewness_couleur    : effet 0.434
      Bénin:  0.007 | Malin:  1.109
   4. couleur_dominante_b : effet 0.419
      Bénin:  0.024 | Malin:  1.115
   5. energie_texture     : effet 0.412
      Bénin:  0.033 | Malin: -1.030
   6. couleur_dominante_g : effet 0.410
      Bénin: -0.985 | Malin:  0.105
   7. variance_locale     : effet 0.399
      Béni

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(



Patient jeune, lésion suspecte:
  • Patient: 28 ans, F, localisation Bras
  • Probabilité de malignité: 64.1%
  • Niveau de risque: ÉLEVÉ
  • Recommandation: Biopsie recommandée
  • Délai: Dans la semaine

Patient âgé, lésion typique:
  • Patient: 67 ans, M, localisation Dos
  • Probabilité de malignité: 38.5%
  • Niveau de risque: MODÉRÉ
  • Recommandation: Surveillance rapprochée
  • Délai: Contrôle dans 3 mois
  • Facteurs de risque: Âge > 50 ans, Sexe masculin, Localisation à risque

Patient moyen, lésion borderline:
  • Patient: 45 ans, F, localisation Jambe
  • Probabilité de malignité: 40.0%
  • Niveau de risque: MODÉRÉ
  • Recommandation: Surveillance rapprochée
  • Délai: Contrôle dans 3 mois

Impact organisationnel sur la dermatologie:
Amélioration de productivité:
  • Diagnostic humain seul: 32 cas/jour
  • Diagnostic assisté IA: 240 cas/jour
  • Amélioration: 650%

Réduction des délais:
  • Délai actuel: 45 jours
  • Nouveau délai: 6 jours
  • Réduction: 39 jours (87%)

Pl

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
